In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

GEMINI_KEY = os.getenv("GEMINI_PAID_KEY")

In [ ]:
from PIL import Image
from typing import TypedDict, List, Optional
from langgraph.graph import StateGraph, END
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    GoogleGenerativeAIEmbeddings
)
from langchain_core.documents import Document
from qdrant_client import QdrantClient
from PIL import Image

/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
gemini = ChatGoogleGenerativeAI(
    model="gemini-3-pro-preview",
    temperature=0,
    tools=["code_execution"],
)

ValidationError: 1 validation error for ChatGoogleGenerativeAI
  Value error, API key required for Gemini Developer API. Provide api_key parameter or set GOOGLE_API_KEY/GEMINI_API_KEY environment variable. [type=value_error, input_value={'model': 'gemini-3-pro-p...: 0, 'model_kwargs': {}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

In [ ]:
embeddings= GoogleGenerativeAIEmbeddings(
    model= "models/mbedding-001"
)

In [ ]:
qdrant= QdrantClient(
    url="http://localhost:6333"
)

## Creating Knowledge Base for RAG

### create topology knowledge chunks (Data Ingestion-> Embedding and Chunking)

In [ ]:
topology_kb=[
    Document(page_content=""" 
    You are a CAD topology extraction expert.
    Analyze the PROVIDED ORIGINAL CAD IMAGE (NOT recolored).

    Output STRICT JSON inside markers.
    No explanation.

    <<<TOPOLOGY_START>>>
    {
    "views": [
        {
        "view": "front",
        "outer_closed_loops": [
            {
            "loop_id": 1,
            "region": "MATERIAL",
            "nodes": 4,
            "edges": 4,
            "edge_type": {
                "straight": 4,
                "chamfer": 0
            }
            }
        ],
        "inner_closed_loops": [
            {
            "loop_id": 2,
            "region": "VOID",
            "nodes": 0,
            "edges": 0,
            "type": "Circle"
            }
        ]
        }
    ]
    }
    <<<TOPOLOGY_END>>>
    """),

    #Text Chunks for explicit rules
    Document(page_content="Inner loops are always VOID, outer loops are always MATERIAL."),
    Document(page_content="If entity is Circle: nodes = 0, edges = 0, only mention the type."),
    Document(page_content="If entity is Slot: only mention the type."),

    #Text chunks for feature-to-topology mapping
    Document(page_content="Rectangle → 4 nodes, 4 straight edges"),
    Document(page_content="Fillet → rounded edge replacing sharp corner"),
    Document(page_content="Chamfer → angled edge replacing corner")

]

#Create qdrant collection
qdrant.recreate_collection(
    collection_name="topology_rules",
    vectors_config={
        "size": 768,
        "distance": "Cosine"
    }
)

#Upload documents as points with embeddings
qdrant.upload_points(
    collection_name="topology_rules",
    points=[
        (
            idx,
            embeddings.embed_query(doc.page_content),
            {"text": doc.page_content}
        )
        for idx, doc in enumerate (topology_kb)
    ]
)
print("Topology knowledge base updated in Qdrant RAG.")

## Langgraph state

In [ ]:
class CADState (TypedDict):
    image_path: str
    visual_fatures: Optional[str]
    decision: Optional[str]
    retrieved_rules: Optional[List[str]]
    topology_output: Optional[str]

### Feature Extraction

Nodel 1

In [ ]:
def extract_features(state: CADState):
    image= Image.open(state["image_path"])

    prompt=""" You are a CAD analyst. Analyze the CAD figure carefully. 

🚨CRITICAL:
1. ⚠️ Ignore the top and bottom tables.
2. ✅ Take the symbols below the table at top right corner.
3. ⚠️ Do not colour the center lines and hidden lines.
4. ⚠️ Do not take numbers as circles (e.g., 0 is a number, not a circle).
5. ⚠️ Do not change the colour of dimension lines; only arrowheads and numbers.
6. ✅ If a slot is present, consider it as a whole slot.  Do not considerit as 2 arcs and 2 lines.
7. ⚠️ Do not miss any chamfers and arcs.

Given a CAD image, change only the colors of existing entities.
Do not modify geometry, scale, position, thickness, or add/remove anything.

Rules (apply only if entity exists):
1. Chamfers (tilted/slanted straight edges) → YELLOW
2. Straight horizontal/vertical lines (BLACK → RED)
3. Arcs (fillets) → LIGHT BLUE
4. Circles → PURPLE
5. Slots → ORANGE
6. Symbols → PINK
7. Dimension arrowheads only → GREEN
8. Dimension text (numbers only) → BROWN
9. Detect separate views and draw a rectangular box around each view (unique color per view)
10. Add a color legend at the top-left listing only colors actually used

Output: ONE image identical to the original except for color changes. The other output is to Return the description as text."""

    response= gemini.invoke[prompt,image]

    visual_description = response.content

    recolored_image= None
    if hasattr(response, "images") and response.images:
        # Load the first returned image
        recolored_image = Image.open(response.images[0].open()) 

    return {
        "visual_features": visual_description,
        "recolored_image": recolored_image
    }

node 2 (decide if we need RAG)

In [ ]:
def decide_topology(state: CADState):
    prompt = f"""
Given the extracted visual features:
{state['visual_features']}

Should topology rules be applied?
Answer ONLY "YES" or "NO".
"""

    response = gemini.invoke(prompt)

    return {
        "decision": response.content.strip()
    }

Node 3 (RAG retrieval)

In [ ]:
def retrieve_topology_rules(state: CADState):
    query_vector= embeddings.embed_query(state["visual_features"]) #query

    hits= qdrant.search(
        collection_name= "topology_rules",
        query_vector=query_vector,
        limit = 3
    )

    rules= [hit.payload["text"] for hit in hits]

    return {
        "retrieved rules": rules  #context to LLM
    }

node 4 (Gemini reasoning)

In [ ]:
def generate_topology(state: CADState):
    # retrieved_rules is the context retrieved from Qdrant
    prompt = f"""
Using:
Visual Features:
{state['visual_features']}

Topology Rules:
{state['retrieved_rules']}

Generate structured topological information in JSON.
Include:
- entity_type
- relationships
- constraints
"""

    response = gemini.invoke(prompt)

    return {
        "topology_output": response.content
    }

In [ ]:
def should_retrieve(state: CADState):
    return state["decision"] == "YES"

## Graph

In [ ]:
graph= StateGraph(CADState)

graph.add_node("vision", extract_features)
graph.add_node("decision", decide_topology)
graph.add_node("rag", retrieve_topology_rules)
graph.add_node("reasoning", generate_topology)

graph.set_entry_point("Start")

graph.add_edge("vision", "decision")
graph.add_conditional_edges(
    "decision",
    should_retrieve,{
        True: "rag",
        False: END
    }
)

graph.add_edge("rag", "reasoning")
graph.add_edge("reasoning", "END")

app=graph.compile()

In [ ]:
result = app.invoke(
    {
        "image_path": ""
    }
)

print("===== TOPOLOGY OUTPUT =====")
print(result["topology_output"])